## Create Assistant
* instructions: 어시스턴트와 모델이 어떻게 행동하거나 응답해야 하는지 알려줍니다.
* model: 미세 조정된 모델을 포함하여 모든 GPT-3.5 또는 GPT-4 모델을 지정할 수 있습니다. 검색 도구에는 gpt-3.5-turbo-1106 및 gpt-4-1106-preview 모델이 필요합니다.
* tools: API는 OpenAI에서 빌드 및 호스팅하는 코드 인터프리터와 웹 검색을 지원합니다. 또한 함수 호출 기능과 유사한 동작으로 사용자 정의 함수 서명을 정의할 수 있습니다

In [3]:
# 패키지 불러오기
import openai
# API 키 지정하여 클라이언트 선언하기
client = openai.OpenAI(api_key = "여기에 api key를 넣어주세요")

## 1. 파일 업로드 및 벡터 저장소 추가하기

In [6]:
# 벡터 저장소 생성하기
vector_store = client.beta.vector_stores.create(name="축구 규칙 파일")

In [7]:
# 파일을 벡터 저장소에 올리기
file_streams = open("축구규칙정리.pdf", "rb")

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=[file_streams]
)
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [8]:
instruction = '''
[목적]
이 GPT는 축구 규칙을 상세히 설명해주는 챗봇입니다.

[규칙]
1. 사용자가 축구 규칙에 대해 질문하면 업로드된 파일에서 해당 내용을 찾아 자세히 답변합니다.
2. 파일안에서 마땅한 답을 찾을 수 없거나 축구 규칙에 관한 질문이 아니면 "축구 규칙에 관한 질문만 부탁해요^^" 라고 답해주세요.
3. 답변의 형태는 아래 예시와 같이 해주세요
예시)
-  질문 : 질문 내용
-  답변 : 답변내용 
4. 모든 질문에 한국어로 답변해주세요.
'''

In [9]:
assistant = client.beta.assistants.create(
  name = "축구 규칙 설명 챗봇",
  instructions=instruction,
  model="gpt-3.5-turbo-1106",
  tools=[{"type": "file_search"}],
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}}
)

print(assistant.id)

asst_gZxQgd3wWma7S0Kmb5nJhRah


In [10]:
print(assistant)

Assistant(id='asst_gZxQgd3wWma7S0Kmb5nJhRah', created_at=1740708095, description=None, instructions='\n[목적]\n이 GPT는 축구 규칙을 상세히 설명해주는 챗봇입니다.\n\n[규칙]\n1. 사용자가 축구 규칙에 대해 질문하면 업로드된 파일에서 해당 내용을 찾아 자세히 답변합니다.\n2. 파일안에서 마땅한 답을 찾을 수 없거나 축구 규칙에 관한 질문이 아니면 "축구 규칙에 관한 질문만 부탁해요^^" 라고 답해주세요.\n3. 답변의 형태는 아래 예시와 같이 해주세요\n예시)\n-  질문 : 질문 내용\n-  답변 : 답변내용 \n4. 모든 질문에 한국어로 답변해주세요.\n', metadata={}, model='gpt-3.5-turbo-1106', name='축구 규칙 설명 챗봇', object='assistant', tools=[FileSearchTool(type='file_search', file_search=FileSearch(max_num_results=None, ranking_options=FileSearchRankingOptions(score_threshold=0.0, ranker='default_2024_08_21')))], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=['vs_67c118e00cf08191b68c41bf448dfee7'])), top_p=1.0, reasoning_effort=None)


## Tread

### Tread 생성하기

In [13]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_YHAEk9Sypyg5oaTYgPAim0BQ', created_at=1740708639, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


### Tread에 메세지 넣기

In [14]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="축구장의 크기는?"
)
print(message)

Message(id='msg_PWUcwke5hZFaIkPAVVdHFG2u', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='축구장의 크기는?'), type='text')], created_at=1740708713, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_YHAEk9Sypyg5oaTYgPAim0BQ')


## Assistant run

In [15]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id)

In [16]:
import time

while run.status not in ["completed", "failed"]:
  run = client.beta.threads.runs.retrieve(
    thread_id= thread.id,
    run_id= run.id
  )
  print(run.status)
  time.sleep(5)

in_progress
in_progress
completed


## After Run complete

In [17]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [18]:
for each in messages:
  print(each.role + ": "+each.content[0].text.value)
  print("=========")

assistant: 축구장의 크기는 국제경기를 위한 기준과 대회 주최 측의 결정에 따라 다를 수 있습니다. 일반적으로 터치라인의 길이는 최소 100m(110 야드)에서 최대 110m(120 야드)이며, 골라인의 길이는 최소 64m(70 야드)에서 최대 75m(80 야드)입니다. 자세한 내용은 아래 링크를 참고해 주세요. 
- 축구규칙정리.pdf (경기규칙 1조)     
user: 축구장의 크기는?


## Add question

In [19]:

# 메세지 추가
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="오프사이드 룰에 대해 설명해줘"
)

# 메세지 리스트 가져오기
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

#메세지 프린트
for each in messages:
  print(each.role + ": "+each.content[0].text.value)
  print("=========")

user: 오프사이드 룰에 대해 설명해줘
assistant: 축구장의 크기는 국제경기를 위한 기준과 대회 주최 측의 결정에 따라 다를 수 있습니다. 일반적으로 터치라인의 길이는 최소 100m(110 야드)에서 최대 110m(120 야드)이며, 골라인의 길이는 최소 64m(70 야드)에서 최대 75m(80 야드)입니다. 자세한 내용은 아래 링크를 참고해 주세요. 
- 축구규칙정리.pdf (경기규칙 1조)     
user: 축구장의 크기는?


In [20]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,)

while run.status not in ["completed", "failed"]:

  run = client.beta.threads.runs.retrieve(
    thread_id= thread.id,
    run_id= run.id)

  print(run.status)

  time.sleep(5)

queued
in_progress
completed


In [21]:
# 메세지 리스트 가져오기
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

#메세지 프린트
for each in messages:
  print(each.role + ": "+each.content[0].text.value)
  print("=========")

assistant: 오프사이드 규칙은 상대팀의 수비수보다 득점 선수가 더 가까이 있을 때 적용됩니다. 선수는 해당 위치에 있고, 볼이 넘어올 때 혹은 팀 동료가 플레이할 때 오프사이드 위치에 있어서 적극적으로 플레이에 관여할 때 반칙으로 간주됩니다. 그러나 오프사이드 위치에 있더라도 해당 선수가 플레이에 관여하지 않는다면 오프사이드 반칙으로 간주되지 않습니다. 자세한 내용은 "축구규칙정리.pdf" 문서의 제11조를 참고하시기 바랍니다. 
user: 오프사이드 룰에 대해 설명해줘
assistant: 축구장의 크기는 국제경기를 위한 기준과 대회 주최 측의 결정에 따라 다를 수 있습니다. 일반적으로 터치라인의 길이는 최소 100m(110 야드)에서 최대 110m(120 야드)이며, 골라인의 길이는 최소 64m(70 야드)에서 최대 75m(80 야드)입니다. 자세한 내용은 아래 링크를 참고해 주세요. 
- 축구규칙정리.pdf (경기규칙 1조)     
user: 축구장의 크기는?


## Assistants 삭제

In [22]:
response = client.beta.assistants.delete(assistant.id)
print(response)

AssistantDeleted(id='asst_gZxQgd3wWma7S0Kmb5nJhRah', deleted=True, object='assistant.deleted')


## File 삭제

In [23]:
# 업로드한 파일 목록 가져오기
file_list = client.files.list()
print(file_list)

SyncCursorPage[FileObject](data=[FileObject(id='file-GHvmG1aE8auFDXVpjf9NTn', bytes=22073717, created_at=1740708074, filename='축구규칙정리.pdf', object='file', purpose='assistants', status='processed', status_details=None, expires_at=None), FileObject(id='file-WxRxWSmDbzRrjGXNcyKbAy', bytes=22073717, created_at=1740707739, filename='축구규칙정리.pdf', object='file', purpose='assistants', status='processed', status_details=None, expires_at=None)], has_more=False, object='list', first_id='file-GHvmG1aE8auFDXVpjf9NTn', last_id='file-WxRxWSmDbzRrjGXNcyKbAy')


In [24]:
# 특정 파일 ID 가져오기
file_id = file_list.data[0].id
print(file_id)

file-GHvmG1aE8auFDXVpjf9NTn


In [25]:
# 업로드 파일 삭제
response = client.files.delete(file_id)
print(response)

FileDeleted(id='file-GHvmG1aE8auFDXVpjf9NTn', deleted=True, object='file')
